# Linear Regression Consulting Project

You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis!

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('CruiseCrewPrediction').getOrCreate()

In [3]:
from google.colab import files
files.upload()

Saving cruise_ship_info.csv to cruise_ship_info.csv


{'cruise_ship_info.csv': b'Ship_name,Cruise_line,Age,Tonnage,passengers,length,cabins,passenger_density,crew\nJourney,Azamara,6,30.276999999999997,6.94,5.94,3.55,42.64,3.55\nQuest,Azamara,6,30.276999999999997,6.94,5.94,3.55,42.64,3.55\nCelebration,Carnival,26,47.262,14.86,7.22,7.43,31.8,6.7\nConquest,Carnival,11,110.0,29.74,9.53,14.88,36.99,19.1\nDestiny,Carnival,17,101.353,26.42,8.92,13.21,38.36,10.0\nEcstasy,Carnival,22,70.367,20.52,8.55,10.2,34.29,9.2\nElation,Carnival,15,70.367,20.52,8.55,10.2,34.29,9.2\nFantasy,Carnival,23,70.367,20.56,8.55,10.22,34.23,9.2\nFascination,Carnival,19,70.367,20.52,8.55,10.2,34.29,9.2\nFreedom,Carnival,6,110.23899999999999,37.0,9.51,14.87,29.79,11.5\nGlory,Carnival,10,110.0,29.74,9.51,14.87,36.99,11.6\nHoliday,Carnival,28,46.052,14.52,7.27,7.26,31.72,6.6\nImagination,Carnival,18,70.367,20.52,8.55,10.2,34.29,9.2\nInspiration,Carnival,17,70.367,20.52,8.55,10.2,34.29,9.2\nLegend,Carnival,11,86.0,21.24,9.63,10.62,40.49,9.3\nLiberty*,Carnival,8,110.0,29.74,

In [4]:
data = spark.read.csv('cruise_ship_info.csv', inferSchema=True, header=True)


In [5]:
data.printSchema()
print(data.head())
data.columns

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)

Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55)


['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew']

In [6]:
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.linalg import Vectors

In [7]:
index = StringIndexer(
    inputCol = "Cruise_line",
    outputCol = "CruiseLine")

indexer = index.fit(data).transform(data)

In [8]:
indexer.printSchema()


root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)
 |-- CruiseLine: double (nullable = false)



In [9]:
indexer.columns


['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'CruiseLine']

In [10]:
assembler = VectorAssembler(
    inputCols = ['passengers', 'length',
                 'cabins', 'passenger_density', 'CruiseLine'],
    outputCol = "features")

output = assembler.transform(indexer)
output.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)
 |-- CruiseLine: double (nullable = false)
 |-- features: vector (nullable = true)



In [11]:
mldata = output.select('features','crew')
train,test = mldata.randomSplit([0.7,0.3])
unlabelled = test.select('features')

In [12]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(labelCol='crew', featuresCol='features')
lrModel = lr.fit(train)

In [13]:
print("Intercept :{}".format(lrModel.intercept))
print("Coefficients :{}".format(lrModel.coefficients))

Intercept :-1.9098968846401871
Coefficients :[-0.11593051308852305,0.46635759316092473,0.8376309500918608,0.011232279124540457,0.03578876926014186]


In [14]:
# testing the model on the test data set

test_results = lrModel.evaluate(test)


In [18]:
# Printing the insights
print("R-Squared :{}".format(test_results.r2))
print("RMSE :{}".format(test_results.rootMeanSquaredError))

R-Squared :0.8529730342202713
RMSE :1.3268825338083892


In [19]:
#deploying model on the unlabelled data
deployresult = lrModel.transform(unlabelled)


In [20]:
deployresult.show()


+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[2.08,4.4,1.04,48...|1.8131679960891804|
|[2.08,4.4,1.04,48...|1.8131679960891804|
|[2.96,5.14,1.48,5...| 2.414941256571369|
|[3.88,5.97,1.94,6...|3.1668238047809765|
|[3.94,4.36,0.88,3...|1.1180591452765891|
|[6.84,5.94,3.42,4...|3.8943950329583763|
|[6.84,5.94,3.42,4...|3.8943950329583763|
|[6.86,5.93,3.44,4...|3.5091411304103297|
|[6.94,5.94,3.55,4...| 4.080864020760135|
|[7.0,7.09,3.54,71...| 4.710487813043638|
|[7.76,6.22,3.86,3...|3.8653299103018104|
|[10.8,7.9,5.5,62....| 6.444842156583782|
|[12.0,7.54,6.0,36...| 5.727781486245373|
|[12.14,7.04,6.07,...| 5.471480195895737|
|[13.44,7.22,6.78,...| 6.200518587041078|
|[14.4,7.77,7.2,43...| 6.674023585885935|
|[15.66,8.23,7.83,...|  7.34223788098852|
|[15.66,8.24,7.83,...| 7.346901456920129|
|[17.48,7.54,8.74,...|  7.37020896562589|
|[17.48,7.54,8.74,...|  7.37020896562589|
+--------------------+------------